<div class="jumbotron jumbotron-fluid">
  <div class="container">
    <p class="h1" style="color:blue; text-align:center">Implémentez un modèle de<br/>scoring </p>

L’entreprise souhaite mettre en œuvre un outil de `scoring crédit` pour calculer la probabilité qu’un client rembourse son crédit, puis classifie le client en `Défaut(D - classe 1)` ou  `Non-Défaut (ND - Classe 0`). Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

L'objectif est donc d'utiliser ces données financières et socio-économiques historiques pour prédire si un client donné sera en mesure de rembourser le prêt accordé. Il s'agit d'une tâche de classification supervisée standard :

- **Supervisé** : les étiquettes sont incluses dans les données d'entraînement et l'objectif est d'entraîner un modèle pour apprendre à prédire les étiquettes à partir des caractéristiques
- **Classification** : La target est une variable binaire, 0 (credit accordé), 1 (credit refusé)


<h2 style="color:green; text-align:left;"> 0. Problématique Business </h2>

Lorsqu'une instituation financière prête de l'argent à une personne, elle prend le risque que cette dernière `ne rembourse pas cet argent dans le délai convenu`. Ce risque est appelé **Risque de Crédit**. Alors avant d'octroyer du crédit, les instituation financière vérifient si la personne qui demandent un prêt sera capable ou pas de le rembourser (`solvabilité`). 

Grâce à des modèles de Machine Learning, les intitutions financières peuvent modéliser la **probabilité de défaut - `PD`**  de paiement  et ainsi attribuer un `score` à chaque nouveau demandeur de crédit : **Credit Scoring**. 

Dans ce projet, nous apprendrons à construire et évaluer un modèle de Machine Learning pour prédire si un demandeur de crédit sera en défaut de paiement ou non. Il s'agit d'un problème de **classification binaire**. 


En générale le défaut est observé en fonction du taux d'endette, plus celui-ci est élevé, plus le probabilité que le client ne rembpurse pas son credit soit élévé. 

Mais dans le cas espèce, l'évènement défaut est déclenché lorsque le client est difficulté de paiement c-à-d qu'il a un retard de paiementde plus de X jours sur au moins une des Y premières échéances.

<h2 style="color:green; text-align:left;"> 1. Mise en Place de l'Environnement de Travail </h2>

#### Import des librairies utiles à l'analyse exploratoire

In [3]:
# Importation des librairies Python nécessaires
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


# Utils 
import os
import time
import missingno as msno

# Librairie de preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Librairie de modélisation
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

#### Format d'affichage des données

In [4]:
# Affichage d'un maximum de les lignes et colonnes
pd.set_option('display.max_column', 200)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 200)

pd.options.display.float_format = '{:, .2f}'.format

#chiffre après la virgule
pd.set_option('display.precision', 3)

# Affichage des erreurs
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def read_datasets(num_rows = None):   
    with timer("Datasets loaded"):
        print("Loading data ... ")
        app_train = pd.read_csv('data/application_train.csv', nrows= num_rows)
        print("\tapplication_train cols_num: {}, obervations: {}".format(len(app_train.columns), app_train.shape[0]))
        app_test = pd.read_csv('data/application_test.csv', nrows= num_rows)
        print("\tapplication_test cols_num: {}, obervations: {}".format(len(app_test.columns), app_test.shape[0]))
        prev_app = pd.read_csv('data/previous_application.csv', nrows = num_rows)
        print("\tprevious_application cols_num: {}, obervations: {}".format(len(prev_app.columns), prev_app.shape[0]))
        bureau = pd.read_csv('data/bureau.csv', nrows = num_rows)
        print("\tbureau cols_num: {}, obervations: {}".format(len(bureau.columns), bureau.shape[0]))
        bur_bal = pd.read_csv('data/bureau_balance.csv', nrows = num_rows)
        print("\tbureau_balance cols_num: {}, obervations: {}".format(len(bur_bal.columns), bur_bal.shape[0]))
        pos_cash_bal = pd.read_csv('data/POS_CASH_balance.csv', nrows = num_rows)
        print("\tPOS_CASH_balance cols_num: {}, obervations: {}".format(len(pos_cash_bal.columns), pos_cash_bal.shape[0]))
        inst_pay = pd.read_csv('data/installments_payments.csv', nrows = num_rows)
        print("\tinstallments_payments cols_num: {}, obervations: {}".format(len(inst_pay.columns), inst_pay.shape[0]))
        cred_card_bal = pd.read_csv('data/credit_card_balance.csv', nrows = num_rows)
        print("\tcredit_card_balance cols_num: {}, obervations: {}".format(len(cred_card_bal.columns), cred_card_bal.shape[0]))
    
        return app_train, app_test, prev_app, bureau, bur_bal, pos_cash_bal, inst_pay, cred_card_bal